### 셀프 어텐션
---
앞에서 살펴본 트랜스포머 구조에서 멀티헤드 어텐션은 ***셀프 어텐션(self attention)*** 이라고도 불립니다. 트랜스포머 경쟁력의 원천은 셀프 어텐션에 있다고들 하는데요, 여기서는 셀프 어텐션을 좀 더 살펴보겠습니다.  

우선 ***어텐션(attention)*** 은 시퀀스 입력에 수행하는 기계학습 방법의 일종인데요, 어텐션은 시퀀스 요소 가운데 중요한 요소에 집중하고 그렇지 않은 요소는 무시해 task 수행 성능을 끌어 올립니다. 어텐션은 기계 번역 과제에 처음 도입됐습니다.

기계 번역에 어텐션을 도입한다면 타깃 언어를 디코딩할 때 소스 언어의 단어 시퀀스 가운데 디코딩에 도움이 되는 단어 위주로 취사선택해서 번역 품질을 끌어올리게 됩니다. 즉, 어텐션은 디코딩할 때 소스 시퀀스 가운데 중요한 요소만 추립니다.  

셀프 어텐션이란, 말 그대로 자신에게 수행하는 어텐션 기법입니다. 입력 시퀀스 가운데 task 수행에 의미 있는 요소들 위주로 정보를 추출한다는 뜻이죠. 그런데 이렇게만 설명하면 너무 알쏭달쏭하니 자연어 처리에서 자주 쓰이는 합성곱 신경망, 순환 신경망 등과 비교해 셀프 어텐션이 어떤점을 목표로 하는지 살펴보겠습니다.

#### 합성곱 신경망과 비교  

***합성곱 신경망(Convolutional Neural Network; CNN)*** 은 합성곱 필터라는 특수한 장치를 이용해 시퀀스의 지역적인 특징을 잡아내는 모델입니다. 자연어는 기본적으로 시퀀스(단어 또는 형태소의 나열)이고 특정 단어를 기준으로 한 주변 문맥이 의미 형성에 중요한 역할을 하므로 CNN이 자연어 처리에 널리 쓰이고 있습니다.  

cf) 합성곱 필터(convolutional filter)란 합성곱 신경망을 구성하는 한 요소입니다. 이 필터는 데이터를 전체적으로 훑으면서 인접 정보를 추출하는 역할을 합니다.

다음 그림은 CNN이 문장을 어떻게 인코딩하는지 나타낸 것입니다.  
합성곱 필터(그림에서 괄호쳐진)가 단어를 하나씩 넘기면서 차례대로 읽어 들이는 걸 알 수 있습니다.

<center><그림 3-17 합성곱 신경망></center>

<center>[어제 카페 갔었어] 거기 사람 많더라  </center>

<center>어제 [카페 갔었어 거기] 사람 많더라  </center>

<center>어제 카페 [갔었어 거기 사람] 많더라  </center>

<center>어제 카페 [갔었어 거기 사람] 많더라  </center>

<center>어제 카페 갔었어 [거기 사람 많더라]  </center>

하지만 CNN은 합성곱 필터 크기를 넘어서는 문맥은 읽어내기 어렵다는 단점이 있습니다.  
예를 들어 필터 크기가 3(3개 단어씩 처리)이라면 4칸 이상 떨어져 있는 단어 사이의 의미는 캐치하기 어렵습니다.

#### 순환 신경망과 비교  

***순환 신경망(Recurrent Neural Network; RNN)*** 역시 시퀀스 정보를 압축하는 데 강점이 있는 구조입니다.  
예를 들어 소스 언어 시퀀스인 **어제**, **카페**, **갔었어**, **거기**, **사람**, **많더라**를 인코딩해야 한다고 가정해 봅시다. 그렇다면 RNN은 다음 그림처럼 소스 시퀀스를 차례대로 처리합니다. 

<center><그림 3-18 순환 신경망></center>  

<center>어제 ➡ 카페 ➡ 갔었어 ➡ 거기 ➡ 사람 ➡ 많더라</center>

하지만 RNN은 시퀀스 길이가 길어질수록 정보 압축에 문제가 발생합니다. 오래전에 입력된 단어는 잊어버리거나, 특정 단어 정보를 과도하게 반영해 전체 정보를 왜곡하는 경우가 자주 생긴다는 것이죠.  

기계 번역을 할 때 RNN을 사용한다면 인코더가 디코더로 넘기는 정보는 소스 시퀀스의 마지막인 **많더라**라는 단어의 의미가 많이 반영될 수밖에 없습니다. RNN은 입력 정보를 차례대로 처리하고 오래전에 읽었던 단어는 잊어버리는 경향이 있기 때문입니다.

#### 어텐션과 비교  

다음 그림을 보면 **cafe**에 대응하는 소스 언어의 단어는 **카페**이고 이는 소스 시퀀스의 초반부에 등장한 상황입니다. **cafe**라는 단어를 디코딩해야 할 때 **카페**를 반드시 참조해야 하는데요, 어텐션이 없는 단순 RNN을 사용하면 워낙 초반에 입력된 단어라 모델이 잊었을 가능성이 크고, 이 때문에 번역 품질이 낮아질 수 있습니다.

<그림 3-19 'cafe'의 어텐션>

어제 카페 갔었어 거기 ... *I* *Went* *to* ***cafe***(얘에서 실선이 뻗어나가는 그림) ...

어텐션은 이러한 문제점을 해결하고자 제안됐습니다. 디코더 쪽 RNN에 어텐션을 추가하는 방식입니다. 어텐션은 디코더가 타깃 시퀀스를 생성할 때 소스 시퀀스 전체에서 어떤 요소에 주목해야 할지를 알려주므로 **카페**가 소스 시퀀스 초반에 등장하거나, 소스 시퀀스의 길이가 길어지더라도 번역 품질이 떨어지는 것을 막을 수 있습니다.  
참고로 앞의 그림에서는 어텐션 기법으로 주목되는 단어에 좀 더 짙고 굵은 실선을 그려 놓았습니다.

#### 특징 및 장점  

셀프 어텐션은 자기 자신에 수행하는 어텐션입니다. 다음 그림을 봅시다.  
입력 시퀀스가 **어제**, **카페**, **갔었어**, **거기**, **사람**, **많더라**일 때 **거기**라는 단어가 어떤 의미를 가지는지 계산하는 상황입니다.

<그림 3-20 '거기'의 셀프 어텐션>  

어제 카페 갔었어 ***거기*** 사람 많더라

잘 학습된 셀프 어텐션 모델이라면 **거기**에 대응하는 장소는 **카페**라는 사실을 알아챌 수 있을 것입니다. 그뿐만 아니라 **거기**는 **갔었어**와도 연관이 있음을 확인할 수 있습니다. 트랜스포머 인코더 블록 내부에서는 이처럼 **거기**라는 단어를 인코딩할 때 **카페**, **갔었어**라는 단어의 의미를 강조해서 반영합니다.  

다음 그림은 입력 시퀀스가 **어제**, **카페**, **갔었어**, **거기**, **사람**, **많더라**일 때 **카페**라는 단어가 어떤 의미를 가지는지 계산하는 상황입니다. 트랜스포머 인코더 블록은 **카페**라는 단어를 인코딩할 때 **거기**, **갔었어**라는 단어의 의미를 다른 단어들보다 더 강하게 반영합니다.

<그림 3-21 '카페'의 셀프 어텐션>  

어제 ***카페*** 갔었어 거기 사람 많더라

셀프 어텐션 수행 대상은 입력 시퀀스 전체입니다. 앞의 두 그림에서 **거기**와 **카페**만을 예로 들었지만 실제로는 **어제-** 전체 입력 시퀀스, **갔었어-** 전체 입력 시퀀스, **사람-** 전체 입력 시퀀스, **많더라-** 전체 입력 시퀀스 모두 어텐션 계산을 수행합니다.  

이처럼 개별 단어와 전체 입력 시퀀스를 대상으로 어텐션 계산을 수행해 문맥 전체를 고려하므로 지역적인 문맥만 보는 CNN보다 강점이 있습니다. 아울러 모든 경우의 수를 고려하기 때문에(단어들 서로가 서로를 1대1로 바라보게 함) 시퀀스 길이가 길어지더라도 정보를 잊거나 왜곡할 염려가 없습니다.  
이는 RNN의 단점을 극복한 지점입니다.

어텐션과 셀프 어텐션의 주요 차이를 살펴보면 다음과 같습니다.  

1. 어텐션은 소스 시퀀스 전체 단어들(**어제**, **카페**, ... ,**많더라**)과 타깃 시퀀스 단어 하나(cafe) 사이를 연결하는 데 쓰입니다. 반면 셀프 어텐션은 입력 시퀀스 전체 단어들(그림 3-20, 3-21) 사이를 연결합니다.  

2. 어텐션은 RNN 구조 위에서 동작하지만 셀프 어텐션은 RNN 없이 동작합니다.  
3. 타깃 언어의 단어를 1개 생성할 때 어텐션은 1회 수행하지만 셀프 어텐션은 인코더, 디코더 볼록의 개수만큼 반복 수행합니다.

참고로 이 절에서는 트랜스포머 인코더 내부에서 동작하는 셀프 어텐션을 예로 들었지만, 트랜스포머 디코더에서 셀프 어텐션이 적용됩니다.

### 계산 예시  

셀프 어텐션은 **쿼리(queury)**, **키(key)**, **밸류(value)** 3가지 요소가 서로 영향을 주고받는 구조입니다.  
트랜스포머 블록에는 문장 내 각 단어가 벡터(vector) 형태로 입력되는데요, 여기서 벡터란 일단 숫자의 나열 정도로 이해해 주면 좋을 것 같습니다.